# RWKV Chains

LLMs are generally trained with different datasets and configurations, making them react differently to the same input. In the case of RWKV, because it is not transformer-based, most prompts built inside LangChain are not compatible with it. Therefore we collected and built a set of prompts in `RWKV_chains` to make LangChain work with RWKV.

## 0. Load a model first

**Please change the `model_name_or_path` to your own model path.**

In [1]:
import os
os.environ["RWKV_JIT_ON"] = '1'
os.environ["RWKV_CUDA_ON"] = '1' # if '1' then use CUDA kernel for seq mode (much faster)

from langchain.llms import RWKV

weight_path = "D:/weights/rwkv/RWKV-4-Raven-7B-v11-Eng99%-Other1%-20230427-ctx8192.pth"
tokenizer_json = "D:/weights/rwkv/20B_tokenizer.json"

model = RWKV(model=weight_path, strategy="cuda fp16i8 *20 -> cuda fp16", tokens_path=tokenizer_json)


Using C:\Users\kinet\AppData\Local\torch_extensions\torch_extensions\Cache\py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file C:\Users\kinet\AppData\Local\torch_extensions\torch_extensions\Cache\py310_cu117\wkv_cuda\build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module wkv_cuda...


CUDA compiled
RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 6

Loading D:/weights/rwkv/RWKV-4-Raven-7B-v11-Eng99%-Other1%-20230427-ctx8192.pth ...
Strategy: (total 32+1=33 layers)
* cuda [float16, uint8], store 20 layers
* cuda [float16, float16], store 13 layers
0-cuda-float16-uint8 1-cuda-float16-uint8 2-cuda-float16-uint8 3-cuda-float16-uint8 4-cuda-float16-uint8 5-cuda-float16-uint8 6-cuda-float16-uint8 7-cuda-float16-uint8 8-cuda-float16-uint8 9-cuda-float16-uint8 10-cuda-float16-uint8 11-cuda-float16-uint8 12-cuda-float16-uint8 13-cuda-float16-uint8 14-cuda-float16-uint8 15-cuda-float16-uint8 16-cuda-float16-uint8 17-cuda-float16-uint8 18-cuda-float16-uint8 19-cuda-float16-uint8 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 25-cuda-float16-float16 26-cuda-float16-float16 27-cuda-float16-float16 28-cuda-float16-float16 29-cuda-float16-float16 30-cuda-float16-float16 31-cuda-float16-float16 32-cuda-float16-float

## A do-all prompt

The Raven versions of RWKV are instruction fined-tuned with Alpaca datasets. Therefore it's much easier if we just include the Alpaca prompt before assigning RWKV to do anything. The Alpaca prompt template is shown below, and we have a `PromptTemplate` called `rwkv_prompt` for ease of use.

```text
Below is an instruction that describes a task. Write a response that appropriately completes the request.
# Instruction:
{Your instruction or question}

# Response:
```



In [2]:
from rwkv_chains import rwkv_prompt
from langchain.chains import LLMChain
chain = LLMChain(llm=model, prompt=rwkv_prompt, verbose=True)

In [3]:
chain.run("Why is fusion energy important?")



> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.
# Instruction:
Why is fusion energy important?

# Response:



c:\Users\kinet\anaconda3\lib\site-packages\rwkv\model.py:625: UserWarning: operator () profile_node %194 : int = prim::profile_ivalue(%192)
 does not have profile information (Triggered internally at C:\cb\pytorch_1000000000000\work\third_party\nvfuser\csrc\graph_fuser.cpp:108.)
  x, state[i*5+0], state[i*5+1], state[i*5+2], state[i*5+3] = ATT(



> Finished chain.


'Fusion energy is important because it has the potential to provide a clean and sustainable source of energy that can replace fossil fuels. It is considered to be one of the most promising sources of energy in the future, as it has the potential to generate electricity without producing greenhouse gases or pollutants.'

## Summarization with RWKV

Inside RWKV, the computation graph resembles RNN in the sense that it has a hidden state that is updated with each token. So in the case of summarization with long texts, the results can be much improved if we put the instruction to summarize **after** the text. The optimized prompt templates can be used via `load_rwkv_summarize_chain`. An example usage is shown below.

In [4]:
from rwkv_chains import load_rwkv_summarize_chain
from langchain.docstore.document import Document

summary_chain = load_rwkv_summarize_chain(llm=model, chain_type="stuff", output_key="summary")
# map_reduce and refine are also available

with open("james_webb_news.txt", "r") as f:
    text = f.read().strip()
docs = [Document(page_content=text)]
result = summary_chain.run(docs)

In [5]:
print(result)


The Smithsonian’s National Air and Space Museum has named the James Webb Space Telescope the 2023 Michael Collins Trophy for Lifetime and Current Achievements.


## Math with RWKV

RWKV is also capable of doing math with external tools, here we showcase how RWKV can call python `numexpr` to do math. The optimized prompt template is at `LLM_MATH_PROMPT`. An example usage is shown below.

In [6]:
from rwkv_chains import LLM_MATH_PROMPT
from langchain.chains import LLMMathChain

math_chain = LLMMathChain(llm=model, prompt=LLM_MATH_PROMPT, verbose=True)

math_chain.run("Calculate 458123412 / 4058912")



> Entering new LLMMathChain chain...
Calculate 458123412 / 4058912

c:\Users\kinet\anaconda3\lib\site-packages\langchain\chains\llm_math\base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(



```text
458123412 / 4058912
```
...numexpr.evaluate("458123412 / 4058912")...

Answer: 112.86852535851972
> Finished chain.


'Answer: 112.86852535851972'

## Bash with RWKV

In [9]:
from rwkv_chains import LLMBashChain

bash_chain = LLMBashChain.from_llm(llm=model, verbose=True)

bash_chain.run("Show the command to use on Windows to print out the current directory")



> Entering new LLMBashChain chain...
Show the command to use on Windows to show current date and time
# Response:
```bash
date -d "now" +"%Y-%m-%d %H:%M:%S"
```
This command will output the current date and time in the format of YYYY-MM-DD HH:MM:SS. To use this command on Windows, you can use the following command:
```bash
cmd /c echo %date% %time% > current_date_time.txt
```
Code: ['date -d "now" +"%Y-%m-%d %H:%M:%S"', 'cmd /c echo %date% %time% > current_date_time.txt']